# Alunos: Gabriel Covello Furlanetto e Evelyn Tenan Ribeiro

# Import de bibliotecas

In [1]:
import os
import pandas as pd
import numpy as np
import json
import joblib
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from statistics import mean


# Extração de dados e formatação de dados

### Variáveis excluídas e motivos: <br> <br>  'product_id' e 'seller_id': excluída por representarem variáveis únicas que não agregam a classificação desejada. <br> <br> 'minimum_quantity', 'views_counts', 'order_counts', 'creation_date': variáveis que não agregam ao modelo pois não possuem relação com os dados que estão sendo classificados, mas sim apenas informações transacionais das buscas

In [2]:
# Leitura do dataset a partir da variável de ambiente
data_path = os.environ['DATASET_PATH']
df = pd.read_csv(data_path)

# Drop de clounas desnecessárias e cujas linhas estão NaN
df = df.drop(['product_id', 'seller_id', 'minimum_quantity', 'view_counts', 'order_counts', 'creation_date'], axis=1)
df=df.dropna()
df.head()

,query,search_page,position,title,concatenated_tags,price,weight,express_delivery,category
0,espirito santo,2,6,Mandala Espírito Santo,mandala mdf,171.890000,1200.0,1,Decoração
1,cartao de visita,2,0,Cartão de Visita,cartao visita panfletos tag adesivos copos lon...,77.670000,8.0,1,Papel e Cia
2,expositor de esmaltes,1,38,Organizador expositor p/ 70 esmaltes,expositor,73.920006,2709.0,1,Outros
3,medidas lencol para berco americano,1,6,Jogo de Lençol Berço Estampado,t jogo lencol menino lencol berco,118.770004,0.0,1,Bebê
4,adesivo box banheiro,3,38,ADESIVO BOX DE BANHEIRO,adesivo box banheiro,191.810000,507.0,1,Decoração


### Pré-processamento de dados que tratará as colunas textuais e irá manter as demais, sem alterações,para que alimentem o modelo. A fim de utilizar-se uma abordagem simples, uma vez que era o primeiro contato dos alunos com NLP, utilizou-se para tratamento de texto o método CountVectorizer, também conhecido como One-hot encoding. <br> <br> Este método cria um vetor que tenha tantas dimensões quanto o número de palavras únicas presentes no dataset. Cada palavra única tem uma dimensão única e será representada por um 1 nessa dimensão e 0s em todas as outras. <br> <br> Ao utilizar esse método, os alunos assumiram o risco de que não seriam realizadas capturas sintáticas e semânticas a respeito dos textos.


In [3]:
# Tratamento de colunas de texto de forma separada, ignorando as demais colunas
preprocess = ColumnTransformer(
    [('query_countvec', CountVectorizer(), 'query'),
     ('title_countvec', CountVectorizer(), 'title'),
     ('concatenated_tags_tfidf', CountVectorizer(), 'concatenated_tags')],
    remainder='passthrough')

### De acordo com alguns padrões utilizados na literatura, 70% dos dados foram deixados para treinamento e 30% foram utilizados para teste

In [4]:
# Definição de X e Y
X = df.drop(['category'], axis=1)
y = df['category']

# Split do dataset entre treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.3)


# Modelagem do problema de classificação

### Nesta etapa foi criado um pipeline para executar o pré-processamento dos dados, padronizar os mesmos na sequência e por fim executar o método classificador, que para este caso foi escolhido o Random Forest Classifier. Este classificador foi escolhido por sua melhor performance em relação ao algoritmo. Também foram testados a Regressão Logística e o Decision Tree Classifier

In [5]:
# Criação do pipeline para realizar o pré-processamento a padronização dos dados e a execução do algoritmo Random Forest
pipeline = Pipeline([
    ('preprocess',preprocess),
    ('normalize', StandardScaler(with_mean=False)),
    ('clf',RandomForestClassifier(n_jobs=-1))
])

# Validação do modelo

### Para validação do modelo, optou-se pela técnica de treinamento com validação cruzada dividindo-se os dados em 5 partes a fim de tentar evitar overfitting <br><br> Além disso, a validação foi feita pelas métricas de acurácia, precisão, recall e F1-Score, que demonstraram sua alta eficiência. 

In [6]:
# Função para execução do modelo
def model_validation(X_train, y_train, X_test, y_test):    
    
    # criação do modelo
    model = pipeline.fit(X_train,y_train)
    
    
    tracc = mean(cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy'))
    trprec = mean(cross_val_score(model, X_train, y_train, cv=5, scoring='precision_weighted'))
    trrec = mean(cross_val_score(model, X_train, y_train, cv=5, scoring='recall_weighted'))
    trf1 = mean(cross_val_score(model, X_train, y_train, cv=5, scoring='f1_weighted'))
    
    
    teacc = accuracy_score(y_test, pipeline.predict(X_test))
    teprec = precision_score(y_test, pipeline.predict(X_test), average='weighted')
    terec = recall_score(y_test, pipeline.predict(X_test), average='weighted')
    tef1 = f1_score(y_test, pipeline.predict(X_test), average='weighted')
    

    result = {
        'train_acc':tracc,
        'test_acc':teacc,
        'train_prec':trprec,
        'test_prec':teprec,
        'train_rec':trrec,
        'test_rec':terec,
        'train_f1':trf1,
        'test_f1':tef1
    }
    
    # Impressão de resultados
    print("Acurácia de treinamento: " + str(result['train_acc']))
    print("Acurácia de teste: " + str(result['test_acc']))
    print("Precisão de treinamento: " + str(result['train_prec']))
    print("Precisão de teste: " + str(result['test_prec']))
    print("Recall de treinamento: " + str(result['train_rec']))
    print("Recall de teste: " + str(result['test_rec']))
    print("F1-Score de treinamento: " + str(result['train_f1']))
    print("F1-Score de teste: " + str(result['test_f1']))
             
    return model, result

# Exportação do modelo

In [7]:
# Execução do modelo e print de resultados
model, result = model_validation(X_train,y_train,X_test,y_test)

# Impressão de métricas em variável de ambiente
metrics_path = os.environ['METRICS_PATH']
with open(metrics_path, 'w') as file:
     file.write(json.dumps(result))

# Impressão de modelo em variável de ambiente
model_path = os.environ['MODEL_PATH']
joblib.dump(model, model_path)


Acurácia de treinamento: 0.8960388177639952
Acurácia de teste: 0.8943068002108593
Precisão de treinamento: 0.8996677006012737
Precisão de teste: 0.8979650406226196
Recall de treinamento: 0.8962271274894342
Recall de teste: 0.8943068002108593
F1-Score de treinamento: 0.8907717413638561
F1-Score de teste: 0.8900320133401923


['/usr/src/data/model.pkl']